In [1]:
import sys
from bs4 import BeautifulSoup
import urllib.request
import requests
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fname = fm.FontProperties(fname='malgun.ttf').get_name()
import sqlite3
from IPython.display import Image
mpl.rc('font',family=fname)
import matplotlib.gridspec as gridspec
#from jupyterthemes import jtplot
#jtplot.style()
%matplotlib inline
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'
import re

In [3]:
df = pd.read_csv('KOSPI_all2.csv')

In [4]:
df

,SIDX,SCODE,SCOMP,SDAY,OPEN_P,HIGH_P,LOW_P,CLOSE_P,VOL
0,20201008095570,95570,AJ네트웍스,20201008,3285,3330,3225,3300,62021
1,20201007095570,95570,AJ네트웍스,20201007,3345,3345,3200,3280,81367
2,20201006095570,95570,AJ네트웍스,20201006,3380,3460,3315,3340,199832
3,20201005095570,95570,AJ네트웍스,20201005,3250,3400,3230,3380,211552
4,20200929095570,95570,AJ네트웍스,20200929,3225,3290,3195,3245,49795
...,...,...,...,...,...,...,...,...,...
212431,20190910003280,3280,흥아해운,20190910,456,465,452,462,115411
212432,20190909003280,3280,흥아해운,20190909,465,471,451,456,137956
212433,20190906003280,3280,흥아해운,20190906,466,469,458,465,163669
212434,20190905003280,3280,흥아해운,20190905,462,467,456,465,325544


In [5]:
# 산업구분 데이터 불러오기
## ref. http://www.wiseindex.com/Index/Index#/WMI500

industry = pd.read_csv('WICS_종목_산업분류.csv',encoding='CP949')

In [6]:
industry

,종목명,대분류,중분류
0,SK이노베이션,에너지,에너지
1,S-Oil,에너지,에너지
2,현대중공업지주,에너지,에너지
3,GS,에너지,에너지
4,SK디스커버리,에너지,에너지
...,...,...,...
2100,한진중공업홀딩스,유틸리티,유틸리티
2101,대성에너지,유틸리티,유틸리티
2102,서울가스,유틸리티,유틸리티
2103,인천도시가스,유틸리티,유틸리티


In [7]:
# 1/20, 9/29 주가 정보만 추출

is_a = df["SDAY"] == 20200929
is_b = df["SDAY"] == 20200120

In [8]:
df_0929 = df[is_a]
df_0120 = df[is_b]

In [9]:
df_0929.head(5)

,SIDX,SCODE,SCOMP,SDAY,OPEN_P,HIGH_P,LOW_P,CLOSE_P,VOL
4,20200929095570,95570,AJ네트웍스,20200929,3225,3290,3195,3245,49795
274,20200929006840,6840,AK홀딩스,20200929,17400,18000,17250,17550,20114
544,20200929027410,27410,BGF,20200929,4095,4100,4030,4080,72477
814,20200929282330,282330,BGF리테일,20200929,121500,124000,120000,123500,71328
1084,20200929138930,138930,BNK금융지주,20200929,5060,5090,5030,5060,936179


In [10]:
df_0120.head(5)

,SIDX,SCODE,SCOMP,SDAY,OPEN_P,HIGH_P,LOW_P,CLOSE_P,VOL
178,20200120095570,95570,AJ네트웍스,20200120,4805,4830,4670,4740,126487
448,20200120006840,6840,AK홀딩스,20200120,34750,35000,34200,34300,16748
718,20200120027410,27410,BGF,20200120,5450,5500,5420,5420,111152
988,20200120282330,282330,BGF리테일,20200120,171000,173000,170000,171000,25568
1258,20200120138930,138930,BNK금융지주,20200120,7190,7300,7190,7220,484725


In [11]:
comp_0929 = df_0929['SCOMP'].to_list()
comp_0929

['AJ네트웍스',
 'AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'BYC',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ씨푸드',
 'CJ제일제당',
 'CS홀딩스',
 'DB',
 'DB금융투자',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'DI동일',
 'DRB동일',
 'DSR',
 'DSR제강',
 'E1',
 'F&F',
 'GKL',
 'GS',
 'GS건설',
 'GS글로벌',
 'GS리테일',
 'HDC',
 'HDC아이콘트롤스',
 'HDC현대EP',
 'HDC현대산업개발',
 'HMM',
 'HSD엔진',
 'IHQ',
 'JB금융지주',
 'JW생명과학',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KCTC',
 'KC그린홀딩스',
 'KC코트렐',
 'KEC',
 'KG동부제철',
 'KG케미칼',
 'KISCO홀딩스',
 'KPX케미칼',
 'KPX홀딩스',
 'KR모터스',
 'KSS해운',
 'KT',
 'KT&G',
 'KTB투자증권',
 'KTcs',
 'KTis',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG헬로비전',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS ELECTRIC',
 'LS네트웍스',
 'LS전선아시아',
 'MH에탄올',
 'NAVER',
 'NHN',
 'NH투자증권',
 'NH프라임리츠',
 'NICE',
 'NI스틸',
 'NPC',
 'OCI',
 'POSCO',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S&T홀딩스',
 'S-Oil',
 'SBS',
 'SBS미디어홀딩스',
 'SG세계물산',
 'SG충방',
 'SH에너지화학',
 'SIMPAC',
 'SJM',
 'SJM홀딩스',
 'SK',
 'SKC',
 'SK가스',
 'SK네트웍스',
 'SK디스커버

In [12]:
comp_0120 = df_0120['SCOMP'].to_list()
comp_0120

['AJ네트웍스',
 'AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'BYC',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ씨푸드',
 'CJ제일제당',
 'CS홀딩스',
 'DB',
 'DB금융투자',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'DI동일',
 'DRB동일',
 'DSR',
 'DSR제강',
 'E1',
 'F&F',
 'GKL',
 'GS',
 'GS건설',
 'GS글로벌',
 'GS리테일',
 'HDC',
 'HDC아이콘트롤스',
 'HDC현대EP',
 'HDC현대산업개발',
 'HMM',
 'HSD엔진',
 'IHQ',
 'JB금융지주',
 'JW생명과학',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KCTC',
 'KC그린홀딩스',
 'KC코트렐',
 'KEC',
 'KG동부제철',
 'KG케미칼',
 'KISCO홀딩스',
 'KPX케미칼',
 'KPX홀딩스',
 'KR모터스',
 'KSS해운',
 'KT',
 'KT&G',
 'KTB투자증권',
 'KTcs',
 'KTis',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG헬로비전',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS ELECTRIC',
 'LS네트웍스',
 'LS전선아시아',
 'MH에탄올',
 'NAVER',
 'NHN',
 'NH투자증권',
 'NH프라임리츠',
 'NICE',
 'NI스틸',
 'NPC',
 'OCI',
 'POSCO',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S&T홀딩스',
 'S-Oil',
 'SBS',
 'SBS미디어홀딩스',
 'SG세계물산',
 'SG충방',
 'SH에너지화학',
 'SIMPAC',
 'SJM',
 'SJM홀딩스',
 'SK',
 'SKC',
 'SK가스',
 'SK네트웍스',
 'SK디스커버

In [13]:
temp = []
for i in comp_0929:
    if i not in comp_0120:
        temp.append(i)

In [14]:
temp

['SK바이오팜',
 '대덕전자',
 '미래에셋맵스리츠',
 '이지스레지던스리츠',
 '이지스밸류리츠',
 '제이알글로벌리츠',
 '케이씨씨글라스',
 '코람코에너지리츠',
 '티와이홀딩스']

In [15]:
containing = df_0929['SCOMP'].str.contains('SK바이오팜|대덕전자|미래에셋맵스리츠|이지스레지던스리츠|이지스밸류리츠|제이알글로벌리츠|케이씨씨글라스|코람코에너지리츠|티와이홀딩스')
df_0929_fin = df_0929[~containing]

In [16]:
df_0929_re = df_0929_fin.reset_index()
df_0929_re.drop(['index'],axis='columns',inplace=True)

In [17]:
df_0929_re

,SIDX,SCODE,SCOMP,SDAY,OPEN_P,HIGH_P,LOW_P,CLOSE_P,VOL
0,20200929095570,95570,AJ네트웍스,20200929,3225,3290,3195,3245,49795
1,20200929006840,6840,AK홀딩스,20200929,17400,18000,17250,17550,20114
2,20200929027410,27410,BGF,20200929,4095,4100,4030,4080,72477
3,20200929282330,282330,BGF리테일,20200929,121500,124000,120000,123500,71328
4,20200929138930,138930,BNK금융지주,20200929,5060,5090,5030,5060,936179
...,...,...,...,...,...,...,...,...,...
781,20200929079980,79980,휴비스,20200929,7560,7770,7480,7620,230018
782,20200929005010,5010,휴스틸,20200929,6630,6690,6620,6680,2393
783,20200929069260,69260,휴켐스,20200929,21500,21500,20400,20600,2564725
784,20200929000540,540,흥국화재,20200929,2670,2840,2645,2715,274386


In [18]:
df_0120_re = df_0120.reset_index()
df_0120_re.drop(['index'],axis='columns',inplace=True)

In [19]:
df_0120_re

,SIDX,SCODE,SCOMP,SDAY,OPEN_P,HIGH_P,LOW_P,CLOSE_P,VOL
0,20200120095570,95570,AJ네트웍스,20200120,4805,4830,4670,4740,126487
1,20200120006840,6840,AK홀딩스,20200120,34750,35000,34200,34300,16748
2,20200120027410,27410,BGF,20200120,5450,5500,5420,5420,111152
3,20200120282330,282330,BGF리테일,20200120,171000,173000,170000,171000,25568
4,20200120138930,138930,BNK금융지주,20200120,7190,7300,7190,7220,484725
...,...,...,...,...,...,...,...,...,...
781,20200120079980,79980,휴비스,20200120,5870,5910,5870,5900,19707
782,20200120005010,5010,휴스틸,20200120,9820,9920,9740,9840,4975
783,20200120069260,69260,휴켐스,20200120,19350,19850,19250,19700,89065
784,20200120000540,540,흥국화재,20200120,3010,3065,2985,3035,10269


In [20]:
rate = []
for i in range(len(df_0929_re)):
    for j in range(len(df_0120_re)):
        if df_0929_re['SCOMP'].iloc[i] == df_0120_re['SCOMP'].iloc[j]:
               result = (df_0929_re['CLOSE_P'][i]-df_0120_re['CLOSE_P'][j])/df_0120_re['CLOSE_P'][j]
               rate.append(result)
               break
        else:
               pass

In [21]:
rate

[-0.3154008438818565,
 -0.48833819241982507,
 -0.24723247232472326,
 -0.2777777777777778,
 -0.29916897506925205,
 -0.02591792656587473,
 -0.11546840958605664,
 -0.3165905631659056,
 0.21993127147766323,
 0.6610800744878957,
 0.5831663326653307,
 -0.06993006993006994,
 -0.14180929095354522,
 -0.06946107784431138,
 -0.0454065469904963,
 0.29310344827586204,
 -0.21233859397417504,
 0.1011378002528445,
 0.18397085610200364,
 0.3055555555555556,
 -0.055853920515574654,
 -0.023706896551724137,
 -0.21497797356828194,
 -0.43249427917620137,
 -0.36344969199178645,
 -0.21207177814029363,
 -0.24660633484162897,
 -0.1475826972010178,
 -0.04932735426008968,
 -0.14976744186046512,
 0.13404255319148936,
 -0.07809110629067245,
 0.9753424657534246,
 0.07582417582417582,
 -0.2136986301369863,
 -0.1471698113207547,
 -0.02564102564102564,
 0.2,
 0.008695652173913044,
 -0.2225672877846791,
 -0.3747323340471092,
 -0.10018552875695733,
 -0.28735632183908044,
 -0.12696148359486448,
 0.2169811320754717,
 0.085

In [22]:
comp_list = df_0120_re['SCOMP']
comp_list

0       AJ네트웍스
1        AK홀딩스
2          BGF
3       BGF리테일
4      BNK금융지주
        ...   
781        휴비스
782        휴스틸
783        휴켐스
784       흥국화재
785       흥아해운
Name: SCOMP, Length: 786, dtype: object

In [23]:
df_rate = pd.DataFrame({'회사명': comp_list,
                       '증감률': rate})

In [24]:
df_rate

,회사명,증감률
0,AJ네트웍스,-0.315401
1,AK홀딩스,-0.488338
2,BGF,-0.247232
3,BGF리테일,-0.277778
4,BNK금융지주,-0.299169
...,...,...
781,휴비스,0.291525
782,휴스틸,-0.321138
783,휴켐스,0.045685
784,흥국화재,-0.105437


In [26]:
df_rate['대분류'] = 0
df_rate['중분류'] = 0

In [27]:
# df_rate에 각 회사별 산업구분 추가

for i in range(len(df_rate)):
    for j in range(len(industry)):
        if df_rate['회사명'].iloc[i] == industry['종목명'].iloc[j]:
            df_rate['대분류'].iloc[i] = industry['대분류'].iloc[j]
            df_rate['중분류'].iloc[i] = industry['중분류'].iloc[j]

C:\Users\starfish\.conda\envs\py37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [28]:
df_rate

,회사명,증감률,대분류,중분류
0,AJ네트웍스,-0.315401,산업재,운송
1,AK홀딩스,-0.488338,소재,소재
2,BGF,-0.247232,경기관련소비재,소매(유통)
3,BGF리테일,-0.277778,경기관련소비재,소매(유통)
4,BNK금융지주,-0.299169,금융,은행
...,...,...,...,...
781,휴비스,0.291525,경기관련소비재,내구소비재와의류
782,휴스틸,-0.321138,소재,소재
783,휴켐스,0.045685,소재,소재
784,흥국화재,-0.105437,금융,보험


In [29]:
for i in range(len(df_rate)):
    if df_rate['대분류'][i]==0:
        print(df_rate['회사명'][i]) # 대분류가 추가되지 않은 회사들

맵스리얼티1
베트남개발1
센트럴인사이트
쌍용차
엘브이엠씨홀딩스
유양디앤유
지코
컨버즈
키위미디어그룹
폴루스바이오팜
한국ANKOR유전
한국패러랠
흥아해운


In [30]:
for i in range(len(df_rate)):
    if df_rate['중분류'][i]==0:
        print(df_rate['회사명'][i]) # 중분류가 추가되지 않은 회사들

맵스리얼티1
베트남개발1
센트럴인사이트
쌍용차
엘브이엠씨홀딩스
유양디앤유
지코
컨버즈
키위미디어그룹
폴루스바이오팜
한국ANKOR유전
한국패러랠
흥아해운


In [34]:
df_rate.to_csv('.../코스피종목_증감률+섹터명.csv',index=False,encoding='utf-8-sig')

In [35]:
# 다시 불러오기

indus = pd.read_csv('코스피종목_증감률+섹터명.csv',encoding='CP949')

In [36]:
indus

,회사명,증감률,대분류,중분류
0,AJ네트웍스,-0.315401,산업재,운송
1,AK홀딩스,-0.488338,소재,소재
2,BGF,-0.247232,경기관련소비재,소매(유통)
3,BGF리테일,-0.277778,경기관련소비재,소매(유통)
4,BNK금융지주,-0.299169,금융,은행
...,...,...,...,...
781,휴비스,0.291525,경기관련소비재,내구소비재와의류
782,휴스틸,-0.321138,소재,소재
783,휴켐스,0.045685,소재,소재
784,흥국화재,-0.105437,금융,보험


In [37]:
indus_worst100 = indus.sort_values(by='증감률',ascending=True)[:100]
indus_best100 = indus.sort_values(by='증감률',ascending=False)[:100]

In [38]:
indus_worst100.to_csv('.../new_코스피종목_증감률_떡락100개.csv',index=False,encoding='utf-8-sig')

In [39]:
indus_best100.to_csv('.../new_코스피종목_증감률_떡상100개.csv',index=False,encoding='utf-8-sig')

In [40]:
industry.to_csv('.../WICS_종목_산업구분.csv',index=False,encoding='utf-8-sig')

In [41]:
indus.to_csv('.../코스피종목_증감률+섹터명.csv',index=False,encoding='utf-8-sig')

In [31]:
df_rate_worst100 = df_rate.sort_values(by='증감률',ascending=True)[:100]
df_rate_best100 = df_rate.sort_values(by='증감률',ascending=False)[:100]

In [32]:
df_rate_worst100.to_csv('.../new_코스피종목_증감률_떡락100개.csv',index=False,encoding='utf-8-sig')

In [33]:
df_rate_best100.to_csv('.../new_코스피종목_증감률_떡상100개.csv',index=False,encoding='utf-8-sig')